# Util Functions

In [1]:
from sentence_transformers import SentenceTransformer
from langchain_core.embeddings import Embeddings
from langchain_chroma import Chroma  # or use FAISS, Pinecone, etc.

class E5Embeddings(Embeddings):
    def __init__(self, model_name="intfloat/multilingual-e5-large"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return self.model.encode(
            texts,
            normalize_embeddings=True
        ).tolist()

    def embed_query(self, text):
        return self.model.encode(
            [f"query: {text}"],
            normalize_embeddings=True
        )[0].tolist()

def create_vector_store(docs_split, embeddings, persist_directory="./chroma_db"):
    vectorstore = Chroma.from_documents(
        documents=docs_split,
        embedding=embeddings,
        persist_directory=persist_directory
    )
    return vectorstore

# E5 models work best with instruction prefixes
def add_e5_prefix_to_docs(docs_split):
    """Add 'passage: ' prefix for E5 models"""
    for doc in docs_split:
        doc.page_content = f"passage: {doc.page_content}"
    return docs_split

c:\Users\yanal\OneDrive\Desktop\Project in ML repo\Project_in_ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create Reviews Vectorstore

In [2]:
import json
import os
from langchain_core.documents import Document

with open("data/cleaned_reviews.json", encoding="utf-8") as f:
    data = json.load(f)

docs = []

for course in data:
    for i, r in enumerate(course["reviews"]):
        docs.append(
            Document(
                page_content=r["content"],
                metadata={
                    "course_id": course["course_id"],
                    "course_name": course["course_name"],
                    "lecturer": course["lecturer"],
                    "rank": r["rank"],
                    "date": r["time"],
                    "review_idx": i
                }
            )
        )

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # Updated import
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap= 50,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

## Some nice custom preprocessing
# documents[0].page_content = documents[0].page_content.replace(". .", "")
docs_split = text_splitter.split_documents(docs)

# assign chunk id
for i, doc in enumerate(docs_split):
    doc.metadata["id"] = i

In [4]:
docs_split = add_e5_prefix_to_docs(docs_split)
embeddings = E5Embeddings()
vectorstore = create_vector_store(docs_split, embeddings)

# Create Query Classification VS

In [5]:
sql_queries = [
    "מה הציון הממוצע בקורס מבני נתונים?",
    "מה הציונים בשנים קודמות של מבני נתונים?",
    "איך השתנה הציון הממוצע בקורס לאורך השנים?",
    "איזה קורסים עם הציון הממוצע הכי נמוך?",
    "דירוג קורסים לפי ממוצע ציונים",
    "מה הציון הממוצע של דני קרן בחדוא 1 בשנה שעברה?",
    "האם יש שיפור בציונים בקורס אלגוריתמים עם משה לוי?",
    "עשרת הממוצעים הכי גבוהים בקורס למידה עמוקה עם ריטה ב 3 שנים אחרונות",
    "מה הקורסים שצריך לפני מבני נתונים?",
    "מהם הקדמים של אלגוריתמים?",
    "איזה קורסים חובה לפני קורס מסוים?",
    "מה צריך ללמוד לפני קורס מערכות הפעלה?",
    "מה הקדמים של מבוא למדעי המחשב?",
    "האם לינארית 1 היא קדם לאנליזה 2?",
    "What is the average grade in Databases?",
    "SELECT the top 5 courses with the highest average grade",
    "How many students passed the AI course?",
    "List all students who failed Operating Systems",
    "מה הציון הממוצע בקורס מבני נתונים?",
    "מה הקורס עם מספר המתקדמים הגבוה ביותר?",
    "List courses that require Calculus as a prerequisite",
    "How many courses require Algebra 2 as prerequisite?",
    "מהם הקורסים המצריכים לאלגוריתמים?",
    "What is the maximum average in Computer Networks?",
    "Show students with grade above 90 in Databases",
    "מה הם הממוצעים המינימליים בקורס מערכות הפעלה?",
    "List all courses that have no prerequisites",
    "What are the prerequisites for reinforcement learning",
]

semantic_queries = [
    "חוות דעת על קורס מבני נתונים",
    "מה הסטודנטים אומרים על קורס למידה עמוקה?",
    "Why do students struggle with Computer Networks?",
    "What do students think about Artificial Intelligence?",
    "מרצים מומלצים",
    "חוות דעת על דני קרן",
    "האם קורס אלגוריתמים קשה?",
    "איך אני יכול לשפר את הציונים שלי בקורסים?",
    "What do students think about Databases course?",
    "Which professor is recommended for AI course?",
    "How difficult is Operating Systems?",
    "מה דעתכם על הקורס מבוא למבני נתונים?",
    "Which courses in the prerequisites table are most interesting?",
    "מה חוות הדעת של הסטודנטים על הקורס למידת מכונה?",
    "Is Computer Networks course more practical than Database Systems?",
    "Which courses require more preparation according to students?",
    "מהם הקורסים הכי מומלצים ללימודי הסמינר?",
    "Explain why the AI course is challenging",
    "How do students rate the Algebra 1 course?",
    "מהן ההמלצות לגבי פרויקט גמר בקורס למידה עמוקה?",
    "Which courses are easier to pass",
    "מה כדאי לדעת לפני שמתחילים את קורס תורת המשחקים?",
    "קורסים עם תרגילי בית קשים",
    "קורסים עם תרגילי בית שצריכים השקעה ומאמץ",
    "חוות דעת על קורס מבוא לחומרה",
    "מרצים טובים ומומלצים",
    "קורסים שמסתכלים עליהם בעבודה",
    "איזה קורס נחשב קשה יותר לפי ביקורות?",
    "מה יותר מומלץ – קורס A או קורס B?",
    "איזה קורס מקבל ביקורות חיוביות יותר?",
    "Compare student satisfaction between two courses",
    "Is Course A more recommended than Course B?",
    "מי נחשב ברור יותר בהסברים?",
    "השוואת סגנון הוראה בין שני מרצים",
    "מי נחשב קשוח יותר בבדיקות?",
    "איזה מרצה נחשב מעביר חומר בצורה מעניינת יותר?",
    "איזה מרצה מועדף על ידי הסטודנטים?",
    "איך הקורס אלגוריתמים שונה אצל מרצה A לעומת מרצה B?",
    "איזה מרצה מלמד את הקורס בצורה קלה יותר להבנה?",
    "Which instructor improves the course experience?",
    "עם מי עדיף לקחת את הקורס X, עם מרצה A או מרצה B?"
]

print("Query lists defined successfully.")

Query lists defined successfully.


In [6]:
classification_documents = []

# SQL 'q' (grades) queries
for query in sql_queries:
    classification_documents.append(
        Document(
            page_content=f"passage: {query}",
            metadata={
                "route": "sql"
            }
        )
    )


# Semantic queries
for query in semantic_queries:
    classification_documents.append(
        Document(
            page_content=f"passage: {query}",
            metadata={
                "route": "semantic"
            }
        )
    )

print(f"Created {len(classification_documents)} classification documents.")

Created 69 classification documents.


In [7]:
classification_vectorstore = create_vector_store(
    classification_documents,
    embeddings,
    persist_directory="./classification_db"
)

print("Classification vector store created at ./classification_db")

Classification vector store created at ./classification_db


# Create Table Router VS

In [8]:
schema_documents = [
  {
    "table": "q",
    "type": "grades",
    "content": "This table stores historical course grade statistics by academic year and semester. It is used to answer questions about grades, average grades, performance trends over years, comparisons between years or semesters, highest or lowest averages and historical grade data. Typical questions include asking about grades in previous years, average grade in a course.",
    "columns": [
      { "col": "id", "description": "Unique identifier for the record" },
      { "col": "course", "description": "Course name" },
      { "col": "lecture", "description": "Lecturer name" },
      { "col": "year", "description": "Academic hebrew year" },
      { "col": "semester", "description": "Semester in which the course was taught(A/B/C)" },
      { "col": "moed", "description": "Exam session(a/b/c/f)" },
      { "col": "avg", "description": "Average grade for the course in that year and semester" },
      { "col": "tag", "description": "Course category(duty/choose)" },
      { "col": "proj", "description": "Whether the course includes a project component" }
      #{ "col": "rank", "description": "Relative ranking of the course by average grade or difficulty" }
    ],
    "example_queries": [
      "מה הציון הממוצע בקורס מבני נתונים",
      "מה הציונים בשנים קודמות של מבני נתונים",
      "איך השתנה הציון הממוצע בקורס לאורך השנים",
      "איזה קורסים עם הציון הממוצע הכי נמוך",
      "דירוג קורסים לפי ממוצע ציונים"
    ],
    "negative_scope": [
      "This table does not contain information about course prerequisites",
      "This table does not describe relationships between courses"
    ]
  },
  {
    "table": "tkdams",
    "type": "prerequisites",
    "content": "This table stores prerequisite (kdam) relationships between courses. It is used to answer questions about which courses must be taken before another course, required background courses, prerequisite chains, and course dependency structures. Typical questions include asking which courses are required before a given course or what prerequisites a course has.",
    "columns": [
      { "col": "name", "description": "Course name" },
      { "col": "kdams", "description": "List of prerequisite courses required before taking the course" }
    ],
    "example_queries": [
      "מה הקורסים שצריך לפני מבני נתונים",
      "מהם הקדמים של אלגוריתמים",
      "איזה קורסים חובה לפני קורס מסוים",
      "מה צריך ללמוד לפני קורס מערכות הפעלה"
    ],
    "negative_scope": [
      "This table does not contain grades or average grade information",
      "This table does not store performance or ranking data"
    ]
  }
]

# embed content + example_queries + negative_scope
# Store table as metadata

schema_docs_for_vectorstore = []
for schema in schema_documents:
    combined_text = schema["content"] + " " + " ".join(schema["example_queries"]) + " " + " ".join(schema["negative_scope"])

    # Prepare metadata for Chroma (ensure simple types or JSON strings)
    #schema["columns"] = json.dumps(schema["columns"]) # Convert list of dicts to string

    schema_docs_for_vectorstore.append(
        Document(
            page_content=combined_text,
            metadata={
                "table_name": schema["table"],
                "table_type": schema["type"],
                #"full_schema": schema["columns"] # store only cols
            }
        )
    )

schema_vectorstore = create_vector_store(
    schema_docs_for_vectorstore,
    embeddings,
    persist_directory="./db_schemas"
)

print("Schema vector store created at ./db_schemas")

Schema vector store created at ./db_schemas
